In [9]:
import os 
dir_path = os.path.dirname(os.path.realpath("__file__"))
os.chdir(dir_path) 

In [10]:
from data_collection import get_sym_data
from data_cleaning import foward_fill, normalize
import database_manager as dm
from pykx import q, QError
os.chdir(dir_path) 


In [11]:
# pull google stock data
sym = "googl"
sym = sym.upper()
table = get_sym_data(sym)
print(table)

Datetime                      Open     High     Low      Close    Volume
------------------------------------------------------------------------
2024.07.02D13:30:00.000000000 182.01   182.63   182      182.54   706507
2024.07.02D13:31:00.000000000 182.52   182.52   182.01   182.03   65513 
2024.07.02D13:32:00.000000000 182.03   182.31   182.03   182.305  68866 
2024.07.02D13:33:00.000000000 182.31   182.4894 182.31   182.35   78758 
2024.07.02D13:34:00.000000000 182.3461 182.3838 182.05   182.11   44688 
2024.07.02D13:35:00.000000000 182.115  182.27   181.9301 181.9301 58004 
2024.07.02D13:36:00.000000000 181.97   182.09   181.63   181.63   83597 
2024.07.02D13:37:00.000000000 181.635  181.8    181.565  181.7    101035
2024.07.02D13:38:00.000000000 181.69   182.07   181.69   181.775  99613 
2024.07.02D13:39:00.000000000 181.7997 181.86   181.7    181.79   74468 
2024.07.02D13:40:00.000000000 181.77   181.9    181.6999 181.79   62158 
2024.07.02D13:41:00.000000000 181.7853 181.85   181

In [12]:
q.set('t',table)
q('count t')

pykx.LongAtom(pykx.q('389'))

In [13]:
q('del:389?2')
q('count del')

pykx.LongAtom(pykx.q('389'))

In [14]:
# simuate missing values
table = q('update Close: ?[del=0; 0Nf; Close] from t')
print(table)

Datetime                      Open     High     Low      Close    Volume
------------------------------------------------------------------------
2024.07.02D13:30:00.000000000 182.01   182.63   182      182.54   706507
2024.07.02D13:31:00.000000000 182.52   182.52   182.01   182.03   65513 
2024.07.02D13:32:00.000000000 182.03   182.31   182.03   182.305  68866 
2024.07.02D13:33:00.000000000 182.31   182.4894 182.31   182.35   78758 
2024.07.02D13:34:00.000000000 182.3461 182.3838 182.05   182.11   44688 
2024.07.02D13:35:00.000000000 182.115  182.27   181.9301 181.9301 58004 
2024.07.02D13:36:00.000000000 181.97   182.09   181.63   181.63   83597 
2024.07.02D13:37:00.000000000 181.635  181.8    181.565           101035
2024.07.02D13:38:00.000000000 181.69   182.07   181.69   181.775  99613 
2024.07.02D13:39:00.000000000 181.7997 181.86   181.7    181.79   74468 
2024.07.02D13:40:00.000000000 181.77   181.9    181.6999          62158 
2024.07.02D13:41:00.000000000 181.7853 181.85   181

In [7]:
# forward fill missing values
table = foward_fill(table)
print(table)

Datetime                      Open     High     Low      Close    Volume
------------------------------------------------------------------------
2024.07.02D13:30:00.000000000 182.01   182.63   182               706507
2024.07.02D13:31:00.000000000 182.52   182.52   182.01            65513 
2024.07.02D13:32:00.000000000 182.03   182.31   182.03            68866 
2024.07.02D13:33:00.000000000 182.31   182.4894 182.31   182.35   78758 
2024.07.02D13:34:00.000000000 182.3461 182.3838 182.05   182.35   44688 
2024.07.02D13:35:00.000000000 182.115  182.27   181.9301 181.9301 58004 
2024.07.02D13:36:00.000000000 181.97   182.09   181.63   181.9301 83597 
2024.07.02D13:37:00.000000000 181.635  181.8    181.565  181.7    101035
2024.07.02D13:38:00.000000000 181.69   182.07   181.69   181.7    99613 
2024.07.02D13:39:00.000000000 181.7997 181.86   181.7    181.79   74468 
2024.07.02D13:40:00.000000000 181.77   181.9    181.6999 181.79   62158 
2024.07.02D13:41:00.000000000 181.7853 181.85   181

In [8]:
# N_normalized price change %
# R_daily return %
# C_culminative returns %
table = normalize(table)
q.set('t',table)
table = q('select Datetime, Open, N_Open, R_Open, C_Open from t')
table

,Datetime,Open,N_Open,R_Open,C_Open
,,,,,
0,2024.07.02D13:30:00.000000000,182.01,1f,0f,0f
1,2024.07.02D13:31:00.000000000,182.52,1.28021,0.2802098,0.2802098
2,2024.07.02D13:32:00.000000000,182.03,1.010991,-0.269219,0.01099075
3,2024.07.02D13:33:00.000000000,182.31,1.164828,0.153837,0.1648278
4,2024.07.02D13:34:00.000000000,182.3461,1.184663,0.01983534,0.1846631
5,2024.07.02D13:35:00.000000000,182.115,1.057695,-0.126968,0.05769518
6,2024.07.02D13:36:00.000000000,181.97,0.9780269,-0.0796683,-0.02197313
7,2024.07.02D13:37:00.000000000,181.635,0.7939674,-0.1840595,-0.2060326
8,2024.07.02D13:38:00.000000000,181.69,0.8241898,0.03022248,-0.1758102
